In [1]:
import argparse
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
import os
from datetime import datetime
from PIL import Image
import io

# ---------------------- FONCTIONS UTILITAIRES ----------------------

def take_full_page_screenshot(driver, filename):
    """Prend une capture d'écran de toute la page."""
    window_size = driver.get_window_size()
    total_height = driver.execute_script("return document.body.parentNode.scrollHeight")
    driver.set_window_size(window_size['width'], total_height)
    time.sleep(2)
    driver.save_screenshot(filename)
    driver.set_window_size(window_size['width'], window_size['height'])

def check_date(target_date):
    """Vérifie si la date cible est dans moins de 7 jours et après aujourd'hui."""
    your_date = datetime.strptime(target_date, "%d-%b-%y")
    today = datetime.now()
    is_after_today = your_date > today
    difference = your_date - today
    days_difference = difference.days
    next_week = days_difference == 6
    less_7_days = days_difference <= 6
    possible = is_after_today & less_7_days
    print(f"Your date: {your_date.strftime('%Y-%m-%d')}")
    print(f"Today: {today.strftime('%Y-%m-%d')}")
    print(f"Is after today: {is_after_today}")
    print(f"Days difference: {days_difference}")
    return possible, next_week

def access_login(website_url, email_login, secret_password):
    """Se connecte au site avec email et mot de passe, prend des captures d'écran."""
    print("🚀 Script simple - Email + Password + Screenshots")
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1400, 900)
    driver.refresh()
    print("🌐 Connexion au site...")
    driver.get(website_url)
    time.sleep(3)
    print(f"📄 Titre: {driver.title}")
    driver.save_screenshot("1_avant_saisie.png")
    print("📸 Screenshot 1: 1_avant_saisie.png")
    email_field = driver.find_element(By.XPATH, "//input[@type='email']")
    if email_field:
        print("📝 Saisie de l'email...")
        email_field.clear()
        email_field.send_keys(email_login)
        time.sleep(1)
    driver.save_screenshot("2_apres_email.png")
    print("📸 Screenshot 2: 2_apres_email.png")
    print("🔑 Recherche du champ Password...")
    password_field = driver.find_element(By.XPATH, "//input[@type='password']")
    if password_field:
        print("📝 Saisie du password...")
        password_field.clear()
        password_field.send_keys(secret_password)
        print("✅ Password saisi")
        time.sleep(1)
    driver.save_screenshot("3_apres_password.png")
    print("📸 Screenshot 3: 3_apres_password.png")
    login_button = driver.find_element(By.XPATH, "//input[@value='Login']")
    login_button.click()
    print("✅ Bouton cliqué!")
    print("⏳ Attente de la redirection...")
    time.sleep(5)
    driver.save_screenshot("4_apres_clic.png")
    print("📸 Screenshot 4: 4_apres_clic.png")
    return driver

def check_basket(driver, login_url):
    """Vérifie si le panier est vide après connexion."""
    if driver.current_url == login_url:
        is_empty = driver.execute_script("return document.querySelector('span.basket-badge').textContent;") == "0"
        return is_empty
    else:
        print("not login")

def click_for_me(driver, target_date, my_name):
    """Clique sur le bouton de réservation pour soi-même."""
    moi_elements = driver.find_elements(By.XPATH, f"//*[contains(text(), '{my_name}')]")
    for i, moi_element in enumerate(moi_elements):
        parent = moi_element.find_element(By.XPATH, "./..")
        moi_buttons = parent.find_elements(By.XPATH, ".//*[contains(text(), 'Book') or contains(text(), 'Réserver') or contains(text(), 'Select') or contains(text(), 'Choose')]")
        if moi_buttons:
            moi_button = moi_buttons[0]
            print(f"✅ Bouton trouvé près de Moi: '{moi_button.text.strip()}'")
            print(f"🖱️ Clic pour réserver pour Moi...")
            moi_button.click()
            print("✅ Réservation Moi cliquée!")
            time.sleep(3)
            driver.save_screenshot(f"7_before_basket_{target_date.replace('-', '_')}.png")
            print(f"📸 Screenshot final: reservation_moi_{target_date.replace('-', '_')}.png")
            return True
    return False

def look_for_slots(driver, target_date, target_name, target_time):
    """Recherche les créneaux disponibles correspondant aux critères."""
    print("🔍 Recherche de boutons avec données de cours...")
    course_buttons = driver.find_elements(By.XPATH, "//button[@data-class-time]")
    print(f"📋 {len(course_buttons)} bouton(s) de cours trouvé(s)")
    found_slots = None
    for i, button in enumerate(course_buttons):
        class_name = button.get_attribute('data-class-name') or ''
        class_date = button.get_attribute('data-class-date') or ''
        class_time = button.get_attribute('data-class-time') or ''
        class_spaces = button.get_attribute('data-class-spaces') or '0'
        class_location = button.get_attribute('data-class-location') or ''
        class_venue = button.get_attribute('data-class-venue') or ''
        date_match = target_date in class_date
        name_match = target_name in class_name
        time_match = target_time in class_time.split("-")[0] or target_time.lstrip() in class_time.split("-")[0]
        if date_match and time_match and name_match:
            print(f"    ✅ CORRESPONDANCE TROUVÉE!")
            spaces_available = int(class_spaces)
            slot_info = {
                'button': button,
                'class_name': class_name,
                'class_date': class_date,
                'class_time': class_time,
                'class_spaces': spaces_available,
                'class_location': class_location,
                'class_venue': class_venue,
                'available': spaces_available > 0
            }
            found_slots = slot_info
            if spaces_available > 0:
                print(f"    ✅ DISPONIBLE - {spaces_available} place(s)")
            else:
                print(f"    ❌ COMPLET - 0 place")
            break
    if found_slots is None:
        print(f"    ❌ NO CORRESPONDANCE TROUVÉE!")
    return found_slots

def click_confirm_basket(driver, target_date):
    """Clique sur le bouton de validation du panier."""
    basket = driver.find_elements(By.XPATH, "//*[contains(text(), 'Checkout')]")
    if basket:
        basket[0].click()
        time.sleep(3)
        driver.save_screenshot(f"8_after_basket_{target_date.replace('-', '_')}.png")
        return True
    return False

def click_on_slot(driver, slot_available):
    """Clique sur le bouton pour réserver le créneau trouvé."""
    info_button = slot_available['button']
    parent = info_button.find_element(By.XPATH, "./..")
    book_buttons = parent.find_elements(By.XPATH, ".//*[contains(text(), 'Book Now') or contains(text(), 'Book')]")
    if book_buttons:
        print(f"✅ Bouton 'Book Now' trouvé dans le même conteneur")
        book_buttons[0].click()
        time.sleep(3)
        driver.save_screenshot("6_click_on_slot.png")
        return True
    return False



In [2]:
email = os.getenv('YOUR_SECRET_EMAIL')
password = os.getenv('YOUR_SECRET_PASSWORD')
logon_url = os.getenv('YOUR_SECRET_LOGON_URL')
planning_url = os.getenv('YOUR_SECRET_PLANNING_URL')
login_url = os.getenv('YOUR_SECRET_LOGIN_URL')
my_name = os.getenv('YOUR_SECRET_My_NAME')

In [36]:
driver.quit()


In [37]:
# Modifiez juste ces valeurs :
target_date = "31-Jul-25"  # ← VOTRE DATE ICI
COURSE_TYPE = "Indoor Pickleball Intermediate"
TARGET_TIME = "6:30"

   # Connexion au site
driver = access_login(logon_url, email, password)
possible_to_book, next_week_booking = check_date(target_date)


🚀 Script simple - Email + Password + Screenshots
🌐 Connexion au site...
📄 Titre: Login - Registration
📸 Screenshot 1: 1_avant_saisie.png
📝 Saisie de l'email...
📸 Screenshot 2: 2_apres_email.png
🔑 Recherche du champ Password...
📝 Saisie du password...
✅ Password saisi
📸 Screenshot 3: 3_apres_password.png
✅ Bouton cliqué!
⏳ Attente de la redirection...
📸 Screenshot 4: 4_apres_clic.png
Your date: 2025-07-31
Today: 2025-07-27
Is after today: True
Days difference: 3


In [38]:
possible_to_book

True

In [39]:
check_basket(driver, login_url)

True

In [33]:

driver.get('https://mycalgaryjcc.com/CJCC/public/Basket')
time.sleep(3)

driver.save_screenshot("basket.png")

True

In [34]:
def cancel_basket_and_confirm(driver):
    """
    Clique sur 'Cancel Basket' puis confirme l'annulation si le bouton de confirmation apparaît.
    """
    try:
        # 1. Cliquer sur le lien 'Cancel Basket'
        cancel_link = driver.find_element(By.XPATH, "//a[@title='Cancel Basket']")
        cancel_link.click()
        print("✅ Lien 'Cancel Basket' cliqué.")
        time.sleep(2)  # Attendre le chargement de la page de confirmation

        # 2. Cliquer sur le bouton de confirmation
        confirm_btn = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Cancel Basket']")
        confirm_btn.click()
        print("✅ Confirmation 'Cancel Basket' cliquée.")
        time.sleep(2)
        return True
    except Exception as e:
        print("❌ Impossible d'annuler le panier :", e)
        return False

In [ ]:
cancel_basket_and_confirm(driver)
time.sleep(3)
driver.save_screenshot("check.png")

def click_continue_shopping(driver):
    """
    Clique sur le bouton 'Continue Shopping' si présent sur la page.
    """
    try:
        continue_btn = driver.find_element(By.XPATH, "//a[@title='Continue Shopping']")
        continue_btn.click()
        print("✅ Bouton 'Continue Shopping' cliqué.")
        time.sleep(2)
        return True
    except Exception as e:
        print("❌ Bouton 'Continue Shopping' non trouvé ou erreur :", e)
        return False
    
click_continue_shopping(driver)
check_basket(driver, login_url)

✅ Lien 'Cancel Basket' cliqué.
✅ Confirmation 'Cancel Basket' cliquée.


True

In [ ]:
  course_type = args.course_type
    target_time = args.target_time

    # Connexion au site
    driver = access_login(logon_url, email, password)
    possible_to_book, next_week_booking = check_date(target_date)

    if possible_to_book:
        if check_basket(driver, login_url):  # Vérifie que le panier est vide
            driver.get(planning_url)  # Va à la page de planning
            if next_week_booking:
                next_week_btn = driver.find_element(By.XPATH, "//input[@value='Next Week']")
                next_week_btn.click()
            time.sleep(3)
            driver.save_screenshot("5_planning_page.png")
            print("📸 Screenshot: planning_page.png")
            slot_available = look_for_slots(driver, target_date, course_type, target_time)
            if slot_available:
                if click_on_slot(driver, slot_available):
                    if click_for_me(driver, target_date, my_name):
                        click_confirm_basket(driver, target_date)
    driver.quit()

if __name__ == "__main__":
    main()